In [39]:
import numpy as np


In [40]:
'''
성능 개선이 문제.
list에서 검색은 루프 탐색보다. in 키워드로 검색하는 것이 3배는 빠름.

쓸데없이 리스트에 추가하는 것 삭제. insert 횟수 감소.

위치관계에 따른 그룹을 직접 알기 위해 group matrix 추가


'''

'\n성능 개선이 문제.\nlist에서 검색은 루프 탐색보다. in 키워드로 검색하는 것이 3배는 빠름.\n\n쓸데없이 리스트에 추가하는 것 삭제. insert 횟수 감소.\n\n위치관계에 따른 그룹을 직접 알기 위해 group matrix 추가\n\n\n'

In [41]:

input = [[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
height = 3
result=15

# input = [[10, 11, 10, 11], [2, 21, 20, 10], [1, 20, 21, 11], [2, 1, 2, 1]]
# height = 1
# result=18

n = len(input)
npinput = np.asarray(input)

In [42]:
# group api
groups = []

# gi=group index. -1=new
# return ; group index
def group_add(i,j, gi):
    global groups
    if gi==-1:
        groups.append([(i,j)])
        gi = len(groups)-1
        print('newgroup:',gi, 'item=', i,j)
    else:
        groups[gi].append((i,j))
        print('add group=',gi, 'item=', i,j)
    return gi

def group_find(i,j):
    if i<0 or j<0:
        return -1
    if i>=n or j>=n:
        return -1
    for gi, g in enumerate(groups):
        for f in g:
            if f[0]==i and f[1]==j:
                return gi
    return -1

# d = l, r, u, d  (left, right, up, down)
def can_move(i,j, d):
    i2,j2 = i,j
    if d=='l': j2 -=1
    if d=='r': j2 +=1
    if d=='u': i2 -=1
    if d=='d': i2 +=1
    if i2<0 or j2<0:
        return False
    if i2>=n or j2>=n:
        return False
    return abs(input[i][j] - input[i2][j2])<=height

def group_merge(gi, gj):
    global groups
    bak = groups[gj][:]
    print('group merge ',gj,'into',gi)
    groups[gi].extend(bak)
    del groups[gj]
    

In [43]:
groups.clear()
print(group_add(0,0,-1))
print(group_add(1,1,0))
print(group_add(2,2,-1))
print(group_add(3,3,1))
print(group_add(4,4,-1))
print(group_add(5,6,2))

print('find=', group_find(5,6))
print('find=', group_find(2,2))

groups

newgroup: 0 item= 0 0
0
add group= 0 item= 1 1
0
newgroup: 1 item= 2 2
1
add group= 1 item= 3 3
1
newgroup: 2 item= 4 4
2
add group= 2 item= 5 6
2
find= -1
find= 1


[[(0, 0), (1, 1)], [(2, 2), (3, 3)], [(4, 4), (5, 6)]]

In [44]:
groups

[[(0, 0), (1, 1)], [(2, 2), (3, 3)], [(4, 4), (5, 6)]]

In [45]:
group_merge(1,2)

group merge  2 into 1


In [46]:
groups

[[(0, 0), (1, 1)], [(2, 2), (3, 3), (4, 4), (5, 6)]]

In [47]:
can_move(0,2,'r')

True

In [48]:
global groups
print(input)
groups.clear()

for i in range(n):
    for j in range(n):
        gi = group_find(i,j)
        if gi==-1:
            gi = group_add(i,j, -1)
        print('check ', i,j, 'group=', gi)
            
        if can_move(i,j,'r'):
            g2 = group_find(i,j+1)
            if g2==-1:
                group_add(i,j+1,gi)
            elif g2==gi:
                pass
            else:
                group_merge(min(gi,g2), max(gi,g2))
                gi = min(gi, g2)

        if can_move(i,j,'d'):
            g2 = group_find(i+1,j)
            if g2==-1:
                group_add(i+1,j,gi)
            elif g2==gi:
                pass
            else:
                group_merge(min(gi,g2), max(gi,g2))

print(groups)


[[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
newgroup: 0 item= 0 0
check  0 0 group= 0
add group= 0 item= 0 1
check  0 1 group= 0
add group= 0 item= 1 1
newgroup: 1 item= 0 2
check  0 2 group= 1
add group= 1 item= 0 3
add group= 1 item= 1 2
check  0 3 group= 1
newgroup: 2 item= 1 0
check  1 0 group= 2
group merge  2 into 0
check  1 1 group= 0
group merge  1 into 0
check  1 2 group= 0
add group= 0 item= 1 3
check  1 3 group= 0
newgroup: 1 item= 2 0
check  2 0 group= 1
add group= 1 item= 2 1
add group= 1 item= 3 0
check  2 1 group= 1
add group= 1 item= 2 2
add group= 1 item= 3 1
check  2 2 group= 1
add group= 1 item= 2 3
add group= 1 item= 3 2
check  2 3 group= 1
check  3 0 group= 1
check  3 1 group= 1
check  3 2 group= 1
newgroup: 2 item= 3 3
check  3 3 group= 2
[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3)], [(2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2)], [(3, 3)]]


In [49]:
groups

[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3)],
 [(2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2)],
 [(3, 3)]]

In [50]:
# 가장 짧은 곳을 연결. merge
bridge=[] # (i1,j1,i2,j2,cost)
while len(groups)>1:
    firstgroup = groups[0]
    tmpb = [] 
    mincost=99999
    mincostidx=0
    mincostgroup=0
    for it in firstgroup:
        cost=-1
        # 다른 그룹과 접하는지 검사. lrud
        # 접한다면 비용 기록. (누구랑, 누구랑 연결할 때 비용)
        gi = group_find(it[0], it[1]-1)
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]-1])
            tmpb.append( (it[0], it[1], it[0], it[1]-1, cost) )
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0], it[1]+1)
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]+1])
            tmpb.append( (it[0], it[1], it[0], it[1]+1, cost) )           
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0]-1, it[1])
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]-1][it[1]])
            tmpb.append( (it[0], it[1], it[0]-1, it[1], cost) ) 
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
        gi = group_find(it[0]+1, it[1])
        if gi>0:
            cost = abs(input[it[0]][it[1]]-input[it[0]+1][it[1]])
            tmpb.append( (it[0], it[1], it[0]+1, it[1], cost) )           
            if mincost>cost:
                mincost=cost
                mincostidx=len(tmpb)-1
                mincostgroup=gi
    print(tmpb)
    print('cost=',mincost, 'idx=',mincostidx, 'group=', mincostgroup)
    # 최소 비용 캐치. 다리 건설.
    bridge.append(tmpb[mincostidx])
    # 1그룹으로 병합
    group_merge(0, mincostgroup)

print(groups)        
print(bridge)

# total cost
total = 0
for x in bridge:
    total += x[4]
print(total)

[(1, 1, 2, 1, 5), (1, 0, 2, 0, 5), (1, 2, 2, 2, 5), (1, 3, 2, 3, 5)]
cost= 5 idx= 0 group= 1
group merge  1 into 0
[(2, 3, 3, 3, 10), (3, 2, 3, 3, 10)]
cost= 10 idx= 0 group= 1
group merge  1 into 0
[[(0, 0), (0, 1), (1, 1), (1, 0), (0, 2), (0, 3), (1, 2), (1, 3), (2, 0), (2, 1), (3, 0), (2, 2), (3, 1), (2, 3), (3, 2), (3, 3)]]
[(1, 1, 2, 1, 5), (2, 3, 3, 3, 10)]
15


In [51]:
np.array([1,2], dtype=np.uint16)

array([1, 2], dtype=uint16)

In [52]:
bdebug=True

In [53]:
import numpy as np


input=[]
n = 0
gheight=0
bdebug=False
# bdebug=True

groupmat = None


# d = l, r, u, d  (left, right, up, down)
def can_move(i,j, d):
    i2,j2 = i,j
    if d=='l': j2 -=1
    if d=='r': j2 +=1
    if d=='u': i2 -=1
    if d=='d': i2 +=1
    if i2<0 or j2<0:
        return False
    if i2>=n or j2>=n:
        return False
    return abs(input[i][j] - input[i2][j2])<=gheight

   
def group_find2(i,j):
    if i<0 or j<0:
        return -1
    if i>=n or j>=n:
        return -1
    return groupmat[i,j]

    
def solution(land, height):
    global input, n, gheight
    global groupmat
    answer=0
    
    input=land
    gheight = height
    n=len(input)

    print("N=", n)
    groupmat = np.zeros((n,n), dtype=np.uint16)
    groupcnt=0
    for i in range(n):
        if bdebug:
            print("progress ", i, "/", n)
            
        for j in range(n):
            gi = groupmat[i,j]
            if gi==0:
                groupcnt+=1
                groupmat[i,j]=groupcnt
                gi = groupcnt

            if can_move(i,j,'r'):
                g2 = groupmat[i,j+1]
                if g2==0:
                    groupmat[i,j+1]=gi
                elif g2==gi:
                    pass
                else:
                    fromg = max(gi,g2)
                    tog = min(gi,g2)
                    groupmat[groupmat==fromg] = tog
                    groupmat[groupmat>fromg] -=1
                    groupcnt-=1
                    gi = tog

            if can_move(i,j,'d'):
                g2 = groupmat[i+1,j]
                if g2==0:
                    groupmat[i+1,j]=gi
                elif g2==gi:
                    pass
                else:
                    fromg = max(gi,g2)
                    tog = min(gi,g2)
                    groupmat[groupmat==fromg] = tog
                    groupmat[groupmat>fromg] -=1
                    groupcnt-=1
                    gi = tog

    if True:
        print('group count=', groupcnt)
        print('groupmat max=', np.max(groupmat))
        
    if bdebug:
        print(groupmat)
        
#     return 0 ; # debug phase1
    if True:
        print('phase1 end.')

    # 가장 짧은 곳을 연결. merge
#     bridge=[] # (i1,j1,i2,j2,cost)

### dtype이 uint16이면 문제가 그룹의 개수가 65535개가 넘으면 안된다.
### 300x300 매트릭스의 모든 아이템이 그룹이 다를 경우 최대 9만개가 된다. 이렇다면 실패함.
# dtype이 uint32면 메모리가 두 배다.

    # 후보 다리. 이 중에 어느 것이 건설될 것이다. 
    # 1그룹과 다른 그릅의 경계포인트가 될 것임.
    allbridge=np.empty((0,6), dtype=np.uint16) 
    
    # 1그룹에 새로 들어온 아이템. 
    newfirstgroup = np.argwhere(groupmat==1)
    # 계속 1그룹으로 합쳐 나간다. 
    while groupcnt>1:
        mincost=99999
        mincostgroup=0
        
        if bdebug:
            print('group count=', groupcnt)
            
        if groupcnt%1000==2:
            print('group count=', groupcnt)
            
        '''
       new incomming이 없을 수도 있다. 
       기존에서 다시 찾아야 함.
       1그룹에서 다른 그룹으로 연결되는 모든 다리 스캔
        '''
        ab = []
        for it in newfirstgroup:
            cost=-1
            # 다른 그룹과 접하는지 검사. lrud
            # 접한다면 비용 기록. (누구랑, 누구랑 연결할 때 비용)
            if it[1]==0:
                gi = -1
            else:
                gi = groupmat[ it[0], it[1]-1 ]
            if gi>1:
                cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]-1])
                ab.append([it[0], it[1], it[0], it[1]-1, gi, cost])
                
            if it[1]+1>=n:
                gi = -1
            else:
                gi = groupmat[ it[0], it[1]+1 ]
            if gi>1:
                cost = abs(input[it[0]][it[1]]-input[it[0]][it[1]+1])
                ab.append([it[0], it[1], it[0], it[1]+1, gi, cost])
                
            if it[0]==0:
                gi = -1
            else:
                gi = groupmat[ it[0]-1, it[1] ]
            if gi>1:
                cost = abs(input[it[0]][it[1]]-input[it[0]-1][it[1]])
                ab.append([it[0], it[1], it[0]-1, it[1], gi, cost])
                
            if it[0]+1>=n:
                gi = -1
            else:
                gi = groupmat[ it[0]+1, it[1] ]
            if gi>1:
                cost = abs(input[it[0]][it[1]]-input[it[0]+1][it[1]])
                ab.append([it[0], it[1], it[0]+1, it[1], gi, cost])
                
        if len(ab)>0:
            allbridge = np.concatenate([allbridge,np.asarray(ab, dtype=np.uint16)])
#             allbridge = np.append(allbridge, np.asarray(ab, dtype=np.uint16), axis=0)

        if bdebug:
            print('allbridge count=', len(allbridge))
            
        del ab
        
        # find minimum cost
        nallbridge = len(allbridge)
        if nallbridge==0:
            print("allbridge is empty! no route! error.")
            return -1
        else:
            idx = np.argmin(allbridge[:,5])
        
        mincost=allbridge[idx,5]
        mincostgroup = allbridge[idx,4]
        tmpb = allbridge[idx]
        
        if bdebug:
            print('min tmpb=',tmpb)
            print('mincost=',mincost, 'mincostgroup=', mincostgroup)
        
        # mincost 그룹은 1그룹으로 병합예정. 따라서 mincost로 가는 다리 모두삭제.
        idxes=np.where(allbridge[:,4]==mincostgroup)[0]
        allbridge = np.delete(allbridge, idxes, axis=0)

        # 최소 비용 캐치. 다리 건설.
#         bridge.append(tmpb)
        answer+=tmpb[5]
        if bdebug:
            print('construct bridge:', tmpb)

        # 1그룹으로 병합
        newfirstgroup = np.argwhere(groupmat==mincostgroup)
        groupmat[groupmat==mincostgroup] = 1
        groupcnt -=1 # 그룹수 감소
        
        #debug
#         if len(np.unique(groupmat))!=groupcnt:
#             print('error! diff group count ', len(np.unique(groupmat)), groupcnt)
#             break
        
        if bdebug:
            print('')

#     if bdebug:
#         print(bridge)
    # total cost
#     total = 0
#     for x in bridge:
#         total += x[5]
#     if bdebug:
#         print('total=', total)
#     answer = total

    return answer

In [54]:
import numpy as np

x=np.zeros((5,5), dtype=int)

In [55]:
x[x==2]=22
x

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [56]:
input = [[1, 4, 8, 10], [5, 5, 5, 5], [10, 10, 10, 10], [10, 10, 10, 20]]
height = 3
result=15
solution(input, height)

N= 4
group count= 3
groupmat max= 3
phase1 end.
group count= 2


15

In [57]:
for i in range(2):
    s=np.random.randint(1,1000)
    print('SEED=',s)
    np.random.seed(s)
    # stress test . max 300x300
    npinput = np.random.rand(100,100)*100
    npinput = np.asarray(npinput, dtype=int)
#     print(npinput)
    %time ans= solution(npinput.tolist(), 20)
    if ans==-1:
        print('fail!')
        break

SEED= 375
N= 100
group count= 3143
groupmat max= 3143
phase1 end.
group count= 3002
group count= 2002
group count= 1002
group count= 2
Wall time: 756 ms
SEED= 877
N= 100
group count= 3194
groupmat max= 3194
phase1 end.
group count= 3002
group count= 2002
group count= 1002
group count= 2
Wall time: 754 ms


In [58]:
# max 300
npinput = np.random.rand(300,300)*100
npinput = np.asarray(npinput, dtype=int)
print(npinput)

# big group
# %time solution(npinput.tolist(), 80) # 1.5s

#  %time solution(npinput.tolist(), 20)
# 22.2s

# many groups
# %time solution(npinput.tolist(), 1) # worst case...fail. dtype size problem. uint16 to uint32!!!

%time solution(npinput.tolist(), 20)


[[30  9 35 ... 28 12 98]
 [58 21 32 ... 97 76  9]
 [98 20 78 ... 62 17 42]
 ...
 [80 51 12 ... 87  0 67]
 [39 14 63 ... 28  7 87]
 [17 52 27 ... 96 26 18]]
N= 300
group count= 27610
groupmat max= 27610
phase1 end.
group count= 27002
group count= 26002
group count= 25002
group count= 24002
group count= 23002
group count= 22002
group count= 21002
group count= 20002
group count= 19002
group count= 18002
group count= 17002
group count= 16002
group count= 15002
group count= 14002
group count= 13002
group count= 12002
group count= 11002
group count= 10002
group count= 9002
group count= 8002
group count= 7002
group count= 6002
group count= 5002
group count= 4002
group count= 3002
group count= 2002
group count= 1002
group count= 2
Wall time: 25.1 s


831526

In [49]:
np.abs(npinput[0]-npinput[0][0])

array([ 0, 55, 12, 69, 42, 33, 62, 80, 37,  3,  6, 69, 29, 61,  4, 55, 67,
       29, 35, 93, 92, 90, 39, 27, 68, 48, 33, 43,  7, 31, 18, 58, 34, 35,
        4, 18, 58, 83, 15,  2, 79, 72, 81, 40,  2, 29, 75, 91, 38, 40, 34,
        2, 50, 93, 79, 34, 21, 72, 13, 42, 13, 26, 41, 37, 33, 60, 41,  1,
        2,  6, 84, 61, 45,  4, 72, 12,  5, 60, 32, 17, 20, 13, 59, 67, 86,
       10,  4, 84, 66, 83, 53,  2, 43, 91, 43, 47, 74, 31,  7, 49, 58, 22,
       24,  7, 60, 82, 53, 36, 63, 35,  9, 87, 19, 48,  3,  2, 41, 83, 34,
       38, 80, 52, 64, 36, 50, 42, 47, 94,  8, 19,  7, 75, 12, 91, 66, 49,
       23, 18, 90, 26, 80, 90,  3, 86, 85, 61, 68,  7, 94,  6, 63,  4, 78,
       11,  5, 56, 14,  3,  0, 40, 57, 76, 29, 92, 52, 37,  1, 45, 66, 73,
        4, 38, 43, 23, 57, 35,  5, 36, 75, 94,  4, 68, 93, 48,  8,  8, 36,
       11, 49, 18, 42, 56, 21, 44, 75, 43, 12,  9, 37, 24, 80, 43, 93, 22,
       83, 23, 87, 93, 37, 27, 32, 83, 82, 50, 73, 89,  2, 73, 74, 70, 43,
       10, 52, 82, 18, 89

In [328]:
np.max(npinput)

99

In [329]:
npinput==23

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

In [108]:
x2=np.argwhere(npinput==23)[0:5]
print(x2)

[[ 0 13]
 [ 0 72]
 [ 0 85]
 [ 1 34]
 [ 1 85]]


In [109]:
for xi in x2:
    print(xi)

[ 0 13]
[ 0 72]
[ 0 85]
[ 1 34]
[ 1 85]


In [122]:
len(x2)

5

In [185]:
all_bridge=[ [1,1,2,1,1,10],[2,1,2,1,2,10], [3,1,2,1,1,10], [4,1,2,1,1,10],[5,1,2,1,2,10], [6,1,2,1,1,10]]
print(all_bridge)

[[1, 1, 2, 1, 1, 10], [2, 1, 2, 1, 2, 10], [3, 1, 2, 1, 1, 10], [4, 1, 2, 1, 1, 10], [5, 1, 2, 1, 2, 10], [6, 1, 2, 1, 1, 10]]


In [186]:
tmpb=[3,1,2,1,1,10]

In [187]:
# mincost 그룹 다리를 삭제. except tmpb
for x in all_bridge:
    if x[4]==1:
        if x==tmpb:
            pass
        else:
            all_bridge.remove(x)
print(all_bridge)

[[2, 1, 2, 1, 2, 10], [3, 1, 2, 1, 1, 10], [5, 1, 2, 1, 2, 10]]


In [188]:
all_bridge=[ [1,1,2,1,1,10],[2,1,2,1,2,10], [3,1,2,1,1,10], [4,1,2,1,1,10],[5,1,2,1,2,10], [6,1,2,1,1,10]]
npa = np.asarray(all_bridge)
print(npa)

[[ 1  1  2  1  1 10]
 [ 2  1  2  1  2 10]
 [ 3  1  2  1  1 10]
 [ 4  1  2  1  1 10]
 [ 5  1  2  1  2 10]
 [ 6  1  2  1  1 10]]


In [189]:
np.argmin(npa[:,5])

0

In [190]:
np.delete(npa, [1,2], axis=0)

array([[ 1,  1,  2,  1,  1, 10],
       [ 4,  1,  2,  1,  1, 10],
       [ 5,  1,  2,  1,  2, 10],
       [ 6,  1,  2,  1,  1, 10]])

In [43]:
npa

array([[ 2,  1,  2,  1,  2, 10],
       [ 3,  1,  2,  1,  1, 10],
       [ 5,  1,  2,  1,  2, 10]])

In [191]:
np.append(npa,[[8,8,9,9,1,2]], axis=0)

array([[ 1,  1,  2,  1,  1, 10],
       [ 2,  1,  2,  1,  2, 10],
       [ 3,  1,  2,  1,  1, 10],
       [ 4,  1,  2,  1,  1, 10],
       [ 5,  1,  2,  1,  2, 10],
       [ 6,  1,  2,  1,  1, 10],
       [ 8,  8,  9,  9,  1,  2]])

In [192]:
npa[:,4]

array([1, 2, 1, 1, 2, 1])

In [193]:
np.where(npa[:,4]==2)

(array([1, 4], dtype=int64),)

In [194]:
a=np.empty((0,6), dtype=int)

In [195]:
a=np.append(a, [[1,2,3,4,5,6]], axis=0)

In [196]:
a

array([[1, 2, 3, 4, 5, 6]])

In [197]:
len(a)

1

In [198]:
npa

array([[ 1,  1,  2,  1,  1, 10],
       [ 2,  1,  2,  1,  2, 10],
       [ 3,  1,  2,  1,  1, 10],
       [ 4,  1,  2,  1,  1, 10],
       [ 5,  1,  2,  1,  2, 10],
       [ 6,  1,  2,  1,  1, 10]])

In [199]:
np.unique(npa)

array([ 1,  2,  3,  4,  5,  6, 10])

In [200]:
np.random.rand(1,10)

array([[0.52237207, 0.03915694, 0.38832822, 0.78586685, 0.4041051 ,
        0.16637724, 0.57769858, 0.16387643, 0.17710494, 0.77878329]])

In [147]:
def f1(cnt):
    k=np.zeros((0,10))
    for i in range(cnt):
        k = np.append(k, np.random.rand(1,10)*100, axis=0)
    print(k.shape)  
%time f1(20000)    

(50000, 10)
Wall time: 34.5 s


In [148]:
def f2(cnt):
    k=np.zeros((0,10))
    tmp=[]
    for i in range(cnt):
        tmp.append( np.random.rand(1,10)*100 )
    k = np.asarray(tmp)
    print(k.shape)  
%time f2(20000)
    

(50000, 1, 10)
Wall time: 155 ms


In [149]:
def f3(cnt):
    k=np.zeros((0,10))
    tmp=[]
    for i in range(cnt):
        k = np.r_[k,np.random.rand(1,10)*100 ]
    print(k.shape)  
%time f3(20000)

(50000, 10)
Wall time: 35.6 s


In [150]:
def f4(cnt):
    k=np.zeros((0,10))
    for i in range(cnt):
        k = np.concatenate([k, np.random.rand(1,10)*100], axis=0)
    print(k.shape)  
%time f4(20000)    

(50000, 10)
Wall time: 34.5 s


In [156]:
k=np.random.rand(500,500)
k2=np.random.rand(500,500)

%time k3=np.concatenate([k,k2], axis=0)

Wall time: 2 ms


In [183]:
k3=np.delete(k3, [3,5,6,10, 30,40,50], axis=0)

In [184]:
k3.shape

(993, 500)